# Web Scrapping GodsUnchained
Nesse notebook vamos fazer um web scrapping da pagina https://gudecks.com/meta/card-rankings para pegar informações sobre as cartas e gerar um documendo csv.

## 1. Programa que pega informações sobre as cartas

In [8]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from time import sleep

options = Options()
options.add_argument('window-size=800,800')

navegador = webdriver.Chrome()
navegador.get('https://gudecks.com/meta/card-rankings')

sleep(10)

but = navegador.find_element(By.XPATH, '//*[@id="root"]/div[3]/button')
but.click()

sleep(0.5)

# Alterando o numero de matchs necessario para selecionar as cartas 
#but_2 = navegador.find_element(By.XPATH, '//*[@id="root"]/div[1]/div/div[2]/div[1]/div[1]/div[7]/div[2]/div/div/div')
#but_2.click()
#navegador.find_element(By.XPATH, '//*[@id="rrs-decksWithCard-menu"]/li[3]').click() # li[1] ou li[2]...

informations = [] # Todas as paginas
for i in range(49):
    informations.append(BeautifulSoup(navegador.page_source, 'html.parser'))
    but = navegador.find_element(By.XPATH, '//*[@id="root"]/div[1]/div/div[2]/div[1]/div[2]/div[2]/div/div[3]/button')
    but.click()
    sleep(0.025)

all_cards = [] 
for site in informations:
    cards = site.findAll('div', attrs={'class':'rt-tr-group'})
    for card in cards:
        little_list = []
        try:
            little_list.append(card.find('div', attrs={'class':'card-ranking-name'}).text) # Nome da carta
        except:
            continue
        little_list.extend([atr.text for atr in card.findAll('div', attrs={'class':'rt-td react-table-cell'})]) #Info das cartas

        all_cards.append(little_list)
    
    

colunas = ['Name', 'God', 'Set', 'Matches', 'In % of Decks', 'Copies', 'Deck win rate', 'Unique Deck WR','Est. Price']
df = pd.DataFrame(all_cards, columns=colunas)
df



,Name,God,Set,Matches,In % of Decks,Copies,Deck win rate,Unique Deck WR,Est. Price
0,Fusing Fleshspawn,Death,Genesis,101,0.2%,1.58,64.4%,64.4%,0.0071
1,That Which Aches,Neutral,Genesis,605,0.1%,1.00,63.6%,63.6%,0.0845
2,Apocalypse Now,Death,Genesis,2.242,3.6%,1.71,63.3%,63.5%,0.1500
3,Sulphuric Rain,Death,Genesis,1.898,3.1%,1.78,62.8%,63.0%,0.0590
4,Injured Hetairoi,Neutral,Trial,111,0.0%,1.24,62.2%,62.2%,0.0002
...,...,...,...,...,...,...,...,...,...
963,Omnipresence,Death,Genesis,180,0.3%,1.69,37.2%,37.2%,0.0004
964,Proud Conscriptor,Nature,Genesis,123,0.1%,1.54,36.6%,36.6%,0.0119
965,Bonepact Hunter,Neutral,Genesis,136,0.0%,1.22,36.0%,36.0%,0.0023
966,Student of Society,Neutral,Trial,180,0.0%,1.24,33.3%,33.1%,0.0005


In [26]:
df.to_csv('godsunchained_cards_informations.csv')